In [47]:
!pip install --upgrade --quiet  langchain-google-genai



In [48]:
# project_id = 'Enter Project ID' 
# location = 'us' 
# processor_id = 'Enter Processor ID' 
processor_version = 'rc' 
# file_path = "filepath.pdf" 
# mime_type = 'application/pdf'

GCP_PROJECT= PROJECT_ID=project_id='my-project-0004-346516'
LOCATION = location = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = processor_id = "c66ce4457bf5af37"  # Create processor in Cloud Console
FILE_PATH =file_path = pdf_file = "./books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf"

# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types
MIME_TYPE =mime_type = "application/pdf"

"gs://my-project-0004-346516/matchingengine"

'gs://my-project-0004-346516/matchingengine'

In [49]:
def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    A function to process a document online using Google Document AI.
    """

    # Define an options dictionary, which includes the API's URL. This is used to connect to Google's Document AI service
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}

    # Create a Document AI client, think of it as our bridge for communicating with Google's services
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # Generate the complete name of the processor
    # You need to first create a processor in the Google Cloud console
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Read in the document you want to analyze (like an image or PDF), and store it in the variable image_content
    with open(file_path, "rb") as image:
        image_content = image.read()

        # Convert the read document into a format that Google Document AI can understand, i.e., a RawDocument object
        raw_document = documentai.RawDocument(
            content=image_content, mime_type=mime_type
        )
        # Create a request, which includes the name of the processor and the document we want to analyze
        request = documentai.ProcessRequest(
            name=resource_name, raw_document=raw_document
        )
        # Send our request and receive the analysis results
        result = documentai_client.process_document(request=request)
        
        print("Document processing complete.")
        # print(f"Text: {document_object.text}")
        
        # Return this analysis result
        return result.document
    
    

# Load Binary Data into Document AI RawDocument Object
# raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
# request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# # Use the Document AI client to process the sample form
# result = docai_client.process_document(request=request)

# document_object = result.document
# print("Document processing complete.")
# print(f"Text: {document_object.text}")

# page_text =document_object.text

In [50]:
def trim_text(text: str): 
    """ Removes spaces and newline characters. """ 
    return text.strip().replace("\n", " ")

### https://cloud.google.com/document-ai/docs/handle-response

In [51]:
from typing import Optional, Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

def print_page_dimensions(dimension: documentai.Document.Page.Dimension) -> None:
    print(f"    Width: {str(dimension.width)}")
    print(f"    Height: {str(dimension.height)}")


def print_detected_langauges(
    detected_languages: Sequence[documentai.Document.Page.DetectedLanguage],
) -> None:
    print("    Detected languages:")
    for lang in detected_languages:
        print(f"        {lang.language_code} ({lang.confidence:.1%} confidence)")


def print_blocks(blocks: Sequence[documentai.Document.Page.Block], text: str) -> None:
    print(f"    {len(blocks)} blocks detected:")
    first_block_text = layout_to_text(blocks[0].layout, text)
    print(f"        First text block: {repr(first_block_text)}")
    last_block_text = layout_to_text(blocks[-1].layout, text)
    print(f"        Last text block: {repr(last_block_text)}")


def print_paragraphs(
    paragraphs: Sequence[documentai.Document.Page.Paragraph], text: str
) -> None:
    print(f"    {len(paragraphs)} paragraphs detected:")
    first_paragraph_text = layout_to_text(paragraphs[0].layout, text)
    print(f"        First paragraph text: {repr(first_paragraph_text)}")

    last_paragraph_text = layout_to_text(paragraphs[-1].layout, text)
    print(f"        Last paragraph text: {repr(last_paragraph_text)}")


def print_lines(lines: Sequence[documentai.Document.Page.Line], text: str) -> None:
    print(f"    {len(lines)} lines detected:")
    first_line_text = layout_to_text(lines[0].layout, text)
    print(f"        First line text: {repr(first_line_text)}")
    last_line_text = layout_to_text(lines[-1].layout, text)
    print(f"        Last line text: {repr(last_line_text)}")


def print_tokens(tokens: Sequence[documentai.Document.Page.Token], text: str) -> None:
    print(f"    {len(tokens)} tokens detected:")
    first_token_text = layout_to_text(tokens[0].layout, text)
    first_token_break_type = tokens[0].detected_break.type_.name
    print(f"        First token text: {repr(first_token_text)}")
    print(f"        First token break type: {repr(first_token_break_type)}")
    if tokens[0].style_info:
        print_style_info(tokens[0].style_info)

    last_token_text = layout_to_text(tokens[-1].layout, text)
    last_token_break_type = tokens[-1].detected_break.type_.name
    print(f"        Last token text: {repr(last_token_text)}")
    print(f"        Last token break type: {repr(last_token_break_type)}")
    if tokens[-1].style_info:
        print_style_info(tokens[-1].style_info)


def print_symbols(
    symbols: Sequence[documentai.Document.Page.Symbol], text: str
) -> None:
    print(f"    {len(symbols)} symbols detected:")
    first_symbol_text = layout_to_text(symbols[0].layout, text)
    print(f"        First symbol text: {repr(first_symbol_text)}")
    last_symbol_text = layout_to_text(symbols[-1].layout, text)
    print(f"        Last symbol text: {repr(last_symbol_text)}")


def print_image_quality_scores(
    image_quality_scores: documentai.Document.Page.ImageQualityScores,
) -> None:
    print(f"    Quality score: {image_quality_scores.quality_score:.1%}")
    print("    Detected defects:")

    for detected_defect in image_quality_scores.detected_defects:
        print(f"        {detected_defect.type_}: {detected_defect.confidence:.1%}")


def print_style_info(style_info: documentai.Document.Page.Token.StyleInfo) -> None:
    """
    Only supported in version `pretrained-ocr-v2.0-2023-06-02`
    """
    print(f"           Font Size: {style_info.font_size}pt")
    print(f"           Font Type: {style_info.font_type}")
    print(f"           Bold: {style_info.bold}")
    print(f"           Italic: {style_info.italic}")
    print(f"           Underlined: {style_info.underlined}")
    print(f"           Handwritten: {style_info.handwritten}")
    print(
        f"           Text Color (RGBa): {style_info.text_color.red}, {style_info.text_color.green}, {style_info.text_color.blue}, {style_info.text_color.alpha}"
    )


def print_visual_elements(
    visual_elements: Sequence[documentai.Document.Page.VisualElement], text: str
) -> None:
    """
    Only supported in version `pretrained-ocr-v2.0-2023-06-02`
    """
    checkboxes = [x for x in visual_elements if "checkbox" in x.type]
    math_symbols = [x for x in visual_elements if x.type == "math_formula"]

    if checkboxes:
        print(f"    {len(checkboxes)} checkboxes detected:")
        print(f"        First checkbox: {repr(checkboxes[0].type)}")
        print(f"        Last checkbox: {repr(checkboxes[-1].type)}")

    if math_symbols:
        print(f"    {len(math_symbols)} math symbols detected:")
        first_math_symbol_text = layout_to_text(math_symbols[0].layout, text)
        print(f"        First math symbol: {repr(first_math_symbol_text)}")


def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
    process_options: Optional[documentai.ProcessOptions] = None,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
        # Only supported for Document OCR processor
        process_options=process_options,
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document


def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )


In [52]:
document = online_process(
    project_id=project_id,
    location=location,
    processor_id=processor_id,
    file_path=file_path,
    mime_type=mime_type,
)

names = []
name_confidence = []
values = []
value_confidence = []

for page in document.pages:
    print(page)
    # for field in page.form_fields:
    #     names.append(trim_text(field.field_name.text_anchor.content))
    #     name_confidence.append(field.field_name.confidence)
    #     values.append(trim_text(field.field_value.text_anchor.content))
    #     value_confidence.append(field.field_value.confidence)

# df = pd.DataFrame(
#     {
#         "Field Name": names,
#         "Field Name Confidence": name_confidence,
#         "Field Value": values,
#         "Field Value Confidence": value_confidence,
#     }
# )

Document processing complete.
page_number: 1
image {
  content: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\006\336\000\000\010\343\010\002\000\000\000\276\273\271\267\000\000 \000IDATx\234\354\335w\234T\325\375?\376\331\002\273\324e\351H\020P:\212\n\306\202\205(\030\261b\000Q\254\030E\215\032c\214\301|?X\023\023\004%j\022?\326\200Q!(\030\243\202Q\004k\020\025\004\224^\004$\002\262tX\330]\330\362\373c\276\331\337~\331\231a+\007\361\371\374#\217a\316\271\357sf\346^\037\344\305\271\367$\025\025\025E\000\000\000\000\000\016\254\344\320\023\000\000\000\000\000\276\217D\223\000\000\000\000@\000\242I\000\000\000\000 \000\321$\000\000\000\000\020\200h\022\000\000\000\000\010@4\t\000\000\000\000\004 \232\004\000\000\000\000\002\020M\002\000\000\000\000\001\210&\001\000\000\000\200\000D\223\000\000\000\000@\000\242I\000\000\000\000 \000\321$\000\000\000\000\020\200h\022\000\000\000\000\010@4\t\000\000\000\000\004 \232\004\000\000\000\000\002\020M\002\000\000\000\000\001\210&\001\000\000\000\20

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [53]:
# for page in document.pages:
#     print(f"Page {page.page_number}:")
#     print_page_dimensions(page.dimension)
#     print_detected_langauges(page.detected_languages)

#     print_blocks(page.blocks, text)
#     print_paragraphs(page.paragraphs, text)
#     print_lines(page.lines, text)
#     print_tokens(page.tokens, text)
    
#     print(page.paragraphs.reverse)
    
# #     if page.symbols:
# #         print_symbols(page.symbols, text)

# #     if page.image_quality_scores:
# #         print_image_quality_scores(page.image_quality_scores)

# #     if page.visual_elements:
# #         print_visual_elements(page.visual_elements, text)

## Using langchain for Doc AI on GCP

In [54]:
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet  google-cloud-documentai-toolbox

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [55]:
# GCS_OUTPUT_PATH = "gs://BUCKET_NAME/FOLDER_PATH"

GCS_OUTPUT_PATH= BUCKET_URI#"gs://tianhaoz-test/saurabh"

# PROCESSOR_NAME = "projects/cloud-llm-preview1/locations/us-central1/processors/96c7b8734e4ddaba"

# PROCESSOR_NAME = 'projects/PROJECT_ID/locations/us/processors/PROCESSOR_ID'

PROCESSOR_NAME = 'projects/255766800726/locations/us/processors/c66ce4457bf5af37'

# endpoint="projects/cloud-llm-preview1/locations/us-central1/publishers/google/models/medlm-large", instances=instances, parameters=parameters, safety_settings=[]


In [56]:
from langchain_community.document_loaders.blob_loaders import Blob
from langchain_community.document_loaders.parsers import DocAIParser

In [57]:
parser = DocAIParser(
    location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
)

In [58]:
# pdf_file = "./books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf"

!gsutil cp ./books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf "$BUCKET_URI"

Copying file://./books/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf [Content-Type=application/pdf]...
/ [1 files][112.1 KiB/112.1 KiB]                                                
Operation completed over 1 objects/112.1 KiB.                                    


In [59]:
BUCKET_URI

'gs://my-project-0004-346516/matchingengine'

In [60]:
blob = Blob(
    path="gs://my-project-0004-346516/matchingengine/Book1_HarryPotter_and_the_Sorcerers_Stone_pg15.pdf"
)

### This below would take a lot of time (about 2-3 mins for a small PDF)

In [61]:
docs = list(parser.lazy_parse(blob))
print(len(docs))



15


In [62]:
page_contents = [doc.page_content for doc in docs]


In [63]:
# docs[0].page_content
page_contents

['CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last people\nyou’d expect to be involved in anything strange or mysterious, because they just\ndidn’t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did have a\nvery large mustache. Mrs. Dursley was thin and blonde and had nearly twice the\nusual amount of neck, which came in very useful as she spent so much of her\ntime craning over garden fences, spying on the neighbors. The Dursleys had a\nsmall\nson called Dudley and in their opinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn’t think they\ncould bear it if anyone found out about the Potters. Mrs. Potter was Mrs.\nDursle

In [64]:
import pandas as pd

# Create a DataFrame from the splitted texts
df = pd.DataFrame({'pagewise_texts': page_contents})

# Add a row number column
df['page_id'] = df.index + 1
# df['pagewise_texts'] = df['pagewise_texts'].page_content


# Print the DataFrame
print(df)

                                       pagewise_texts  page_id
0   CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...        1
1   something peculiar — a cat reading a map. For ...        2
2   “\n— yes, their son, Harry —”\nMr. Dursley sto...        3
3   learned a new word (“Won’t!”). Mr. Dursley tri...        4
4   agree.”\nHe didn’t say another word on the sub...        5
5   nearest street lamp went out with a little pop...        6
6   “It certainly seems so,” said Dumbledore. “We ...        7
7   Professor McGonagall’s voice trembled as she w...        8
8   see how much better off he’ll be, growing up a...        9
9   “Even if I could, I wouldn’t. Scars can come i...       10
10  happen. Harry Potter rolled over inside his bl...       11
11    HP 1 - Harry Potter and the\nSorcerer's Stone\n       12
12  CHAPTER TWO\nTHE VANISHING GLASS\nN early ten ...       13
13  pulling a spider off one of them, put them on....       14
14  “Darling, you haven’t counted Auntie Marge’s p...  

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size=100
chunk_overlap=20

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators= ["/,", "##", ">", "We"],#'\n\n', '\n'],
    length_function=len,
    is_separator_regex=False,
)

# splitted_texts = text_splitter.create_documents([page_text])
# print("one \n",splitted_texts[0])
# print("two \n",splitted_texts[1])

def split_text_chunks(text, chunk_size):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]


In [66]:
splitted_texts = []

for index, row in df.iterrows():
    splitted_texts += text_splitter.create_documents(row['pagewise_texts'])
    splitted_texts2 = text_splitter.split_text(row['pagewise_texts'])
    print(splitted_texts2)

    # df['splitted_texts'] = splitted_texts2
    
    df['splitted_texts'] = df['pagewise_texts'].apply(lambda x: text_splitter.split_text(x))


['CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last people\nyou’d expect to be involved in anything strange or mysterious, because they just\ndidn’t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did have a\nvery large mustache. Mrs. Dursley was thin and blonde and had nearly twice the\nusual amount of neck, which came in very useful as she spent so much of her\ntime craning over garden fences, spying on the neighbors. The Dursleys had a\nsmall\nson called Dudley and in their opinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn’t think they\ncould bear it if anyone found out about the Potters. Mrs. Potter was Mrs.\nDursle

In [67]:
df.head()

,pagewise_texts,page_id,splitted_texts
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,[CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs....
1,something peculiar — a cat reading a map. For ...,2,[something peculiar — a cat reading a map. For...
2,"“\n— yes, their son, Harry —”\nMr. Dursley sto...",3,"[“\n— yes, their son, Harry —”\nMr. Dursley st..."
3,learned a new word (“Won’t!”). Mr. Dursley tri...,4,[learned a new word (“Won’t!”). Mr. Dursley tr...
4,agree.”\nHe didn’t say another word on the sub...,5,[agree.”\nHe didn’t say another word on the su...


In [68]:
df_exploded = df.explode('splitted_texts')


In [69]:
df_exploded.head()

,pagewise_texts,page_id,splitted_texts
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...
1,something peculiar — a cat reading a map. For ...,2,something peculiar — a cat reading a map. For ...
2,"“\n— yes, their son, Harry —”\nMr. Dursley sto...",3,"“\n— yes, their son, Harry —”\nMr. Dursley sto..."
3,learned a new word (“Won’t!”). Mr. Dursley tri...,4,learned a new word (“Won’t!”). Mr. Dursley tri...
3,learned a new word (“Won’t!”). Mr. Dursley tri...,4,"Well, Ted,” said the weatherman, “I don’t know..."


In [70]:
len(splitted_texts)

26862

In [71]:
df_exploded['splitted_texts_chunks'] = df_exploded['splitted_texts'].apply(lambda x: split_text_chunks(x,chunk_size))


In [72]:
df_exploded_2 = df_exploded.explode('splitted_texts_chunks')


In [73]:
df_exploded_2.head(10)

,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"\nthat they were perfectly normal, thank you v..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"involved in anything strange or mysterious, be..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,rsley was the director of a firm called Grunni...
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"ardly any neck, although he did have a\nvery l..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"nearly twice the\nusual amount of neck, which..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"raning over garden fences, spying on the neigh..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,n their opinion there was no finer boy anywher...
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"lso had a secret, and\ntheir greatest fear was..."
0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,y\ncould bear it if anyone found out about the...


In [76]:
# df_exploded_2 = df_exploded_2.rename(columns={'page_id': 'id'})

df_exploded_2x = df_exploded_2.copy()

df_exploded_2x = df_exploded_2x.reindex()
df_exploded_2x = df_exploded_2x.reset_index()

df_exploded_2x['id'] = df_exploded_2x.index
df_exploded_2x['id'] = df_exploded_2x['id'].astype(str)

# df_exploded_2a = df_exploded_2a.rename(columns={'splitted_texts_chunks_emb': 'embedding'})

df_exploded_2 = df_exploded_2x.copy()

In [77]:
df_exploded_2.head()

,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id
0,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,0
1,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"\nthat they were perfectly normal, thank you v...",1
2,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"involved in anything strange or mysterious, be...",2
3,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,rsley was the director of a firm called Grunni...,3
4,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"ardly any neck, although he did have a\nvery l...",4


In [80]:
%pip install --upgrade --quiet  langchain-core langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [81]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="models/embedding-003")

text = "This is a test document."

query_result = embeddings.embed_query(text)
# print(query_result)

In [82]:
# df = pd.DataFrame({"id": [0], "embedding": [[0.03090338036417961, -0.0172298401594162]]})


In [83]:
# x=embeddings.embed_query(df_exploded_2a.loc[1, 'splitted_texts_chunks'])
# print(x)

#### Convert text columns to embeddings

In [84]:

# for i, row in df_exploded_2a.iterrows():

#     print(i, '  --  ',row['splitted_texts_chunks'])
    
#     emb = embeddings.embed_query(df_exploded_2a.loc[i, 'splitted_texts_chunks'])
#     emb_val = ','.join(map(str, emb))
#     df_exploded_2a.loc[i, 'splitted_texts_chunks_emb'] = emb_val
    
#     # df_exploded_2a.loc[i, 'splitted_texts_chunks_emb_list'] = emb

    
# df_exploded_2a['id'] = df_exploded_2a.index
# df_exploded_2a = df_exploded_2a.rename(columns={'splitted_texts_chunks_emb': 'embedding'})


NameError: name 'df_exploded_2a' is not defined

In [ ]:
# df_exploded_2a[['id','splitted_texts_chunks','embedding']].head()

In [ ]:
# # save id and embedding as a json file
# df_exploded_2a['id'] = df_exploded_2a['id'].astype(str)


# jsonl_string = df_exploded_2a[["id",'splitted_texts_chunks',"embedding"]].to_json(orient="records", lines=True)
# with open("questions.json", "w") as f:
#     f.write(jsonl_string)

# # show the first few lines of the json file
# ! head -n 3 questions.json

In [ ]:
# import json

# # Assuming df_exploded_2a is a pandas DataFrame

# # Convert the "embedding" column to a list of floats
# df_exploded_2a["embedding"] = df_exploded_2a["embedding"].apply(lambda x: list(map(float, x.split(','))))

# # Convert DataFrame to JSON with the desired format
# jsonl_list = df_exploded_2a[["id","embedding"]].to_dict(orient="records")

# # Convert the list of dicts to the desired output format
# output_list = [{"id":str(item["id"]), "embedding":item["embedding"]} for item in jsonl_list]

# # Write the output to a JSON file
# with open("questions.json", "w") as f:
#     for item in output_list:
#         f.write(json.dumps(item) + "\n")
# ! head -n 3 questions.json

In [ ]:
# import json

# # Your input JSON string

# # with open("questions.json", "w") as f:
# #     for item in output_list:
# #         print(item)
# #         item_nospace = {key: value.replace(' ', '') if isinstance(value, str) else value for key, value in item.items()}
# #         f.write(json.dumps(item_nospace) + "\n")

# with open("questions.json", "w") as f:
#     for item in output_list:
#          embedding = item["embedding"].replace(' ', '')
#          f.write(json.dumps(question) + "\n")

# # print(json_string_no_spaces)
# # # Re-encode the dictionary to a formatted JSON string
# # formatted_json_string = json.dumps(data, separators=(',', ':'), indent=2)

# # print(formatted_json_string)
# ! head -n 3 questions.json

### Add embedding to the splitted texts chunk column
### Convert the dataframe into json files required

In [86]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery

QUESTIONS_SIZE = df_exploded_2.shape[0] #1000

bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions`
        where Score > 0 ORDER BY View_Count desc) AS q
        LIMIT {limit} ;
        """
query = QUERY_TEMPLATE.format(limit=QUESTIONS_SIZE)
query_job = bq_client.query(query)
rows = query_job.result()
# df = rows.to_dataframe()

# examine the data
# df.head()

In [87]:
len(df_exploded_2.splitted_texts_chunks)

343

In [88]:
df_exploded_2.head()

,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id
0,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,0
1,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"\nthat they were perfectly normal, thank you v...",1
2,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"involved in anything strange or mysterious, be...",2
3,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,rsley was the director of a firm called Grunni...,3
4,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"ardly any neck, although he did have a\nvery l...",4


In [89]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs
# The following code will get embedding for the question titles and add them as a new column embedding to the DataFrame. This will take a few minutes.

# get embeddings for the question titles and add them as "embedding" column
df = df_exploded_2.assign(embedding=get_embeddings_wrapper(list(df_exploded_2.splitted_texts_chunks)))
df.head()

100%|██████████| 69/69 [01:13<00:00,  1.06s/it]


,index,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks,id,embedding
0,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,0,"[0.003317659953609109, -0.04790826886892319, -..."
1,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"\nthat they were perfectly normal, thank you v...",1,"[0.013395305722951889, -0.08007742464542389, -..."
2,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"involved in anything strange or mysterious, be...",2,"[0.020101193338632584, -0.035850077867507935, ..."
3,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,rsley was the director of a firm called Grunni...,3,"[0.016535211354494095, -0.014058639295399189, ..."
4,0,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,1,CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. ...,"ardly any neck, although he did have a\nvery l...",4,"[0.008282989263534546, -0.046268247067928314, ..."


In [90]:
df_exploded_2.to_csv('df_exploded_2.csv', index=False)


In [91]:
# save id and embedding as a json file
jsonl_string = df[["id",'splitted_texts_chunks', "embedding"]].to_json(orient="records", lines=True)
with open("questions_test.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 questions_test.json

{"id":"0","splitted_texts_chunks":"CHAPTER ONE\nTHE BOY WHO LIVED\nM r. and Mrs. Dursley, of number four, Privet Drive, were proud to say","embedding":[0.00331766,-0.0479082689,-0.0160974599,0.0116275474,0.0583026372,0.0035153672,0.010107493,-0.0088840397,0.0022986734,0.0063093845,0.0147030614,0.0444023684,0.0337760076,-0.0061602024,0.0160618052,0.0424549431,0.0199845526,0.0161699541,-0.0075055822,-0.0430095233,-0.0260914303,0.0205040853,-0.0246986803,0.0146389743,0.0188306961,-0.0562081747,0.030479867,-0.1032175571,-0.0202636141,0.020874992,-0.0294418689,0.0250715278,-0.0764802545,-0.0115942517,0.0315131135,-0.0704317316,0.0127692958,0.0468112305,-0.0004749426,0.0534682795,0.0005832114,-0.004775404,-0.0113134282,0.0188628975,-0.0156529769,0.0383772999,-0.0436384566,0.0189775527,0.0181709286,-0.0362215303,0.0535730608,0.012083495,0.0190793332,-0.0438452885,-0.0129255848,-0.0609461665,0.0359720252,-0.0106597738,-0.0882806331,0.0098421164,-0.0236698724,0.0302701108,-0.0531803966,0.045661

In [92]:
! head -n 3 product-embs.json

{"id":"11863","name":"wacoal women's embrace lace chemise - 814191","embedding":[0.024530868977308273,-0.040793128311634064,0.00047576811630278826,0.017147192731499672,0.026629658415913582,-0.06141049787402153,0.020689481869339943,0.032609399408102036,-0.043242260813713074,-0.0030736858025193214,0.029859514907002449,-0.0070092626847326756,0.0074279718101024628,0.0044401995837688446,0.012789646163582802,0.0056940736249089241,-0.050746820867061615,-0.016834203153848648,-0.0025733353104442358,0.030782531946897507,-0.028840042650699615,0.013945236802101135,-0.0078231878578662872,-0.073326528072357178,-0.028099022805690765,-0.091063208878040314,0.016907420009374619,-0.0062174019403755665,0.0086391577497124672,-0.021324845030903816,-0.030394084751605988,-0.042661987245082855,-0.021932870149612427,-0.024862896651029587,0.025864196941256523,0.0095110638067126274,-0.023645631968975067,0.018030215054750443,0.013453914783895016,0.014843377284705639,-0.021920621395111084,0.0013795378617942333,-0.0

In [93]:
# # save id and embedding as a json file
# jsonl_string = df_exploded_2a[["id", "embedding"]].to_json(orient="records", lines=True)
# with open("questions.json", "w") as f:
#     f.write(jsonl_string)

# # show the first few lines of the json file
# ! head -n 3 questions.json

### Upload the Json File to matching engine

In [94]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

BUCKET_URI_ME="gs://my-project-0004-346516/matchingengine/embedding"
LOCATION = 'us-central1'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [95]:
! gsutil cp questions_test.json {BUCKET_URI_ME}


Copying file://questions_test.json [Content-Type=application/json]...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [96]:
! gsutil ls {BUCKET_URI_ME}

gs://my-project-0004-346516/matchingengine/embedding/
gs://my-project-0004-346516/matchingengine/embedding/questions_test.json


In [97]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-feature-index-{UID}",
    contents_delta_uri=BUCKET_URI_ME,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/255766800726/locations/us-central1/indexes/5427228977120935936/operations/8056447186516312064
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/255766800726/locations/us-central1/indexes/5427228977120935936
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/255766800726/locations/us-central1/indexes/5427228977120935936')


#### Create Index Endpoint and deploy the Index
To use the Index, you need to create an Index Endpoint. It works as a server instance accepting query requests for your Index.


In [98]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-feature-index-endpoint-{UID}", public_endpoint_enabled=True
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912/operations/7395262466222981120
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912')


In [99]:
DEPLOYED_INDEX_ID = f"vs_feature_deployed_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912/operations/1475562231021174784
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912


resource name: projects/255766800726/locations/us-central1/indexEndpoints/3519110109999398912

### Go to you vertex AI console and check the index is CREATED successfully 

### Github matching engine deployment Test

In [ ]:
# ! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" "$BUCKET_URI_ME"


In [ ]:
# ! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests


In [ ]:
# # create Index
# my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
#     display_name=f"vs-test-index-{UID}",
#     contents_delta_uri=BUCKET_URI,
#     dimensions=768,
#     approximate_neighbors_count=10,
#     project = PROJECT_ID
# )

In [ ]:
# # create IndexEndpoint
# my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
#     display_name=f"vs-test-index-endpoint-{UID}", public_endpoint_enabled=True
# )

In [ ]:
!gcloud projects list

### Deploy instead using langchiain

In [ ]:
# PROJECT_ID = "my-project-0004-346516"
# REGION = "us-central1"
# VPC_NETWORK = "matching-engine-vpc"
# PEERING_RANGE_NAME = "ann-langchain-me-range"  # Name for creating the VPC peering.
# # BUCKET_URI = "gs://<bucket_uri>"
# BUCKET_URI="gs://tianhaoz-test"

# # The number of dimensions for the tensorflow universal sentence encoder its 500
# # If other embedder is used, the dimensions would probably need to change. for gecko its 768
# DIMENSIONS = 768
# DISPLAY_NAME = "index-test-name-new"
# EMBEDDING_DIR = f"{BUCKET_URI}/saurabh"
# DEPLOYED_INDEX_ID = "endpoint-test-name"

# PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
# PROJECT_NUMBER = PROJECT_NUMBER[0]
# VPC_NETWORK_FULL = f"projects/{PROJECT_NUMBER}/global/networks/{VPC_NETWORK}"

# CREATE_VPC = True

In [ ]:
# # Remove the if condition to run the encapsulated code
# if CREATE_VPC:
#     # Create a VPC network
#     ! gcloud compute networks create {VPC_NETWORK} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

#     # Add necessary firewall rules
#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-icmp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow icmp

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-internal --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-rdp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-ssh --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:22

#     # Reserve IP range
#     ! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={VPC_NETWORK} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range"

#     # Set up peering with service networking
#     # Your account must have the "Compute Network Admin" role to run the following.
#     ! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={VPC_NETWORK} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

In [ ]:
# my_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
#     display_name=DISPLAY_NAME,
#     contents_delta_uri=EMBEDDING_DIR,
#     dimensions=DIMENSIONS,
#     # approximate_neighbors_count=150,
#     distance_measure_type="DOT_PRODUCT_DISTANCE",
# )

In [ ]:
# my_index = aiplatform.MatchingEngineIndex('projects/801452371447/locations/us-central1/indexes/280810871687479296')
